In [1]:
from map import Map
from algoviz import AlgoViz
from algoviz.svg import SVGView, Circle, Rect, Text
from algoviz.gui import GUI, Button, Label, TextInput
import random, math
from player import Player

def start_game():
    
    l=1000  # spielfeldgröße
    
    view = SVGView(l,l)
    
    running = True
    feld = Rect(0,0,l,l,view)
    feld.set_fill("white")
    
    level = Map(10,10)
    level.create_maze()
    level.zeichnen(view,l)
    
    pl = Player(l//2, l//2, 30, view)
    
    while running == True:
        
        key = view.pressed_key()
        #print("-------------")
        #print(key)
        #print(".")
        #print(view.pressed_key())
        diag = 1 / math.sqrt(2)
        dx, dy = 0, 0
        if key == "s":
            dx, dy = 0, 1
        elif key == "w":
            dx, dy = 0, -1
        elif key == "a":
            dx, dy = -1, 0
        elif key == "d":
            dx, dy = 1, 0
        elif key == "Q":          # oben-links
            dx, dy = -diag, -diag
        elif key == "E":          # oben-rechts
            dx, dy = diag, -diag
        elif key == "A":          # unten-links
            dx, dy = -diag, diag
        elif key == "D":          # unten-rechts
            dx, dy = diag, diag

        elif key == "p":
            game_end()
            running = False
        pl.set_dx(dx * pl.get_speed())
        pl.set_dy(dy * pl.get_speed())

        pl.bewegen()
        
        view.sync()


In [2]:
start_game()

NameError: name 'game_end' is not defined